In [1]:
!pip install implicit

     |████████████████████████████████| 1.1MB 2.8MB/s 

  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420107 sha256=c8309a723f348646b0cf7178580ff2475b51aace34e337bb9d18fc761a884b53
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
from google.colab import drive
drive = drive.mount('data', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at data


None


In [4]:

#path = drive.GetContentFile('retail_train.csv')
data = pd.read_csv('/content/data/My Drive/data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [ ]:
data.tail(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0
2396803,1167,41656790510,663,6430664,1,8.00,3385,0.00,1059,95,0.0,0.0


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [6]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()

popularity['sales_sum'] = np.log(popularity.sales_value)
popularity['sales_sum']=  popularity['sales_sum'].apply(lambda x: '0.1' if x <0.1 else x).astype(float)
popularity['weight'] = popularity.sales_sum/popularity.sales_sum.sum()
popularity['weight_sum'] = 0
popularity['weight_sum'].iloc[0]= popularity['weight'].iloc[0]
for row in range(1, popularity['item_id'].count() ):
  popularity['weight_sum'].iloc[row]= popularity['weight'].iloc[row] + popularity['weight_sum'].iloc[row-1]
popularity.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,item_id,sales_value,sales_sum,weight,weight_sum
0,25671,20.94,3.041661,1.297448e-05,0.000013
1,26081,0.99,0.100000,4.265591e-07,0.000013
2,26093,1.59,0.463734,1.978100e-06,0.000015
3,26190,1.54,0.431782,1.841807e-06,0.000017
4,26355,1.98,0.683097,2.913812e-06,0.000020


In [7]:
popularity

,item_id,sales_value,sales_sum,weight,weight_sum
0,25671,20.94,3.041661,1.297448e-05,0.000013
1,26081,0.99,0.100000,4.265591e-07,0.000013
2,26093,1.59,0.463734,1.978100e-06,0.000015
3,26190,1.54,0.431782,1.841807e-06,0.000017
4,26355,1.98,0.683097,2.913812e-06,0.000020
...,...,...,...,...,...
89046,17991689,2.49,0.912283,3.891425e-06,0.999963
89047,17991691,2.49,0.912283,3.891425e-06,0.999967
89048,18000012,19.96,2.993730,1.277003e-05,0.999980
89049,18024155,3.99,1.383791,5.902687e-06,0.999985


In [9]:
import random
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights['item_id'].values
    weights = items_weights['weight'].values
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [10]:
%%time

# your_code
weighted_random_recommendation(popularity, 5)


CPU times: user 3.41 ms, sys: 77 µs, total: 3.49 ms
Wall time: 3 ms


[9858767, 1008483, 16830379, 1100375, 12171222]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [11]:
result = pd.read_csv('/content/data/My Drive/data/predictions_basic.csv') 
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1000965, 10121560, 9878622, 13512901, 13039531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [12]:
#Поставлю везде между товарами запятые
result['actual'] = result['actual'].apply(lambda x: x.strip('[]').split())
result['random_recommendation'] = result['random_recommendation'].apply(
    lambda x: x.strip('[]').split(sep=', '))
result['popular_recommendation'] = result['popular_recommendation'].apply(
    lambda x: x.strip('[]').split(sep=', '))
result['itemitem'] = result['itemitem'].apply(lambda x: x.strip('[]').split(
    sep=', '))
result['cosine'] = result['cosine'].apply(lambda x: x.strip('[]').split(
    sep=', '))
result['tfidf'] = result['tfidf'].apply(lambda x: x.strip('[]').split(sep=', '))
result['own_purchases'] = result['own_purchases'].apply(
    lambda x: x.strip('[]').split(sep=', '))

In [13]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1000965, 10121560, 9878622, 13512901, 13039531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [17]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
        
    return precision
  
  
result['precision_rndm_recom'] = [precision_at_k(
    result['random_recommendation'][x], result['actual'][x], k=5) 
    for x in result.index]

result_rrecom = (result['precision_rndm_recom'].sum() / 
    len(result['precision_rndm_recom']))
result_rrecom
  


0.0005876591576885408

In [19]:
result['precision_pop_recom'] = [precision_at_k(
    result['popular_recommendation'][x], result['actual'][x], k=5) 
    for x in result.index]

result_poprecom = (result['precision_pop_recom'].sum() / 
    len(result['precision_pop_recom']))
result_poprecom


0.15523996082272282

In [21]:
result['precision_item_recom'] = [precision_at_k(
    result['itemitem'][x], result['actual'][x], k=5) 
    for x in result.index]

result_itemrecom = (result['precision_item_recom'].sum() / 
    len(result['precision_pop_recom']))
result_itemrecom

0.15132223310479923

In [22]:
result['precision_cosine_recom'] = [precision_at_k(
    result['cosine'][x], result['actual'][x], k=5) 
    for x in result.index]

result_cosinerecom = (result['precision_cosine_recom'].sum() / 
    len(result['precision_pop_recom']))
result_cosinerecom

0.155435847208619

In [24]:
result['precision_tfidf_recom'] = [precision_at_k(
    result['tfidf'][x], result['actual'][x], k=5) 
    for x in result.index]

result_tfidfrecom = (result['precision_tfidf_recom'].sum() / 
    len(result['precision_tfidf_recom']))
result_tfidfrecom

0.16121449559255635

In [26]:
result['precision_own_purchases_recom'] = [precision_at_k(
    result['own_purchases'][x], result['actual'][x], k=5) 
    for x in result.index]

result_own_purchasesrecom = (result['precision_own_purchases_recom'].sum() / 
    len(result['precision_own_purchases_recom']))
result_own_purchasesrecom

0.20122428991185112

Наилучший результат показывает алгоритм own_purchases 0,201

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ- {2000, 5000, 10000} товаров. Постройте график зависимости precision@5 от топ-*
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$. Постройте график зависимости precision@5 от $K$ 

In [ ]:
# your_code